# **Downloader and Transcriptor Youtube Video**

## **Download Youtube video**

In [ ]:
!pip install pytube ffmpeg-python

In [ ]:
from pytube import YouTube
import ffmpeg
import os

video_url = input("Enter the youtube video url: ")

try:
    # Crear objeto YouTube
    yt = YouTube(video_url)

    # Obtener el stream de audio
    audio_stream = yt.streams.filter(only_audio=True).first()

    # Descargar el audio
    filename = yt.title
    print(f"Descargando audio de: {filename}")
    audio_file = audio_stream.download(output_path='/content')
except Exception as e:
    print(f"Error al descargar el archivo: {str(e)}")

try:
    # Cambiar la extensión a mp3
    base, ext = os.path.splitext(audio_file)
    mp3_file = base + '.mp3'

    # Convertir a MP3 usando ffmpeg
    print("Convirtiendo a MP3...")
    (
        ffmpeg
        .input(audio_file)
        .output(mp3_file, acodec='libmp3lame', audio_bitrate='128k')
        .overwrite_output()
        .run(quiet=True)
    )

    # Eliminar el archivo original
    os.remove(audio_file)

    audio_file = mp3_file
    filename = os.path.basename(audio_file)[:-4]

    print(f"Descarga y conversión completadas: {filename}")
except Exception as e:
    print(f"Error al convertir: {str(e)}")

Enter the youtube video url: https://www.youtube.com/watch?v=y4SLh1cpap4
Descargando audio de: Non-Technical Intro to Generative AI
Convirtiendo a MP3...
Descarga y conversión completadas: /content/Non-Technical Intro to Generative AI.mp3


## **Transcript File**

In [ ]:
!pip install git+https://github.com/m-bain/whisperx.git --upgrade

  Cloning https://github.com/m-bain/whisperx.git to /tmp/pip-req-build-rl1kp0lp
  Running command git clone --filter=blob:none --quiet https://github.com/m-bain/whisperx.git /tmp/pip-req-build-rl1kp0lp
  Resolved https://github.com/m-bain/whisperx.git to commit f2da2f858e99e4211fe4f64b5f2938b007827e17
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.7/208.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.3/192.3 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━

In [ ]:
import whisperx
import gc

device = "cuda"
batch_size = 16
compute_type = "float16"

# Transcribir
model = whisperx.load_model("large-v2", device, compute_type=compute_type)
audio = whisperx.load_audio(audio_file)
result = model.transcribe(audio, batch_size=batch_size)

# Alinear
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.80k [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

No language specified, language will be first be detected for each audio file (increases inference time).


100%|█████████████████████████████████████| 16.9M/16.9M [00:01<00:00, 10.7MiB/s]
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.3.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.3.0+cu121. Bad things might happen unless you revert torch to 1.x.
Detected language: en (1.00) in first 30s of audio...


Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960_asr_ls960.pth" to /root/.cache/torch/hub/checkpoints/wav2vec2_fairseq_base_ls960_asr_ls960.pth
100%|██████████| 360M/360M [00:05<00:00, 67.6MB/s]


In [ ]:
import re

# Crear marcador para identificar el final del texto
end_marker = "END_TEXT_MARKER"
transcription_text = "\n".join([segment['text'] for segment in result["segments"]])
text_to_traduce = f"{transcription_text} {end_marker}"

words = re.findall(r'\S+', text_to_traduce)
chunks = []
actual_chunk = []
word_counter = 0

# Se dividen en chunks el texto para su posterior traducción
for word in words:
    actual_chunk.append(word)
    word_counter += 1

    # Si se llega al límite de palabras o encontramos el marcador final
    if word_counter >= 2000 or word == end_marker:
        chunks.append(' '.join(actual_chunk))
        actual_chunk = []
        word_counter = 0

# Se añade el último fragmento si queda un sobrante
if actual_chunk:
    chunks.append(' '.join(actual_chunk))

In [ ]:
print(chunks[-1])

LLM OS maybe you have not taken any course so this is going to help you to see how the future of LLMOS is coming and what led us up to there. And let me know in the comment section, if you like this kind of content, I'll put together more. This took me a lot of time to create the framework design, put it in a particular thought process to, you know, make it, make it understandable. And this is basically what a lot of LLM courses offer. So I'm, I'm definitely looking forward to hear more feedback. And if you like this kind of format, subscribe to the channel. See you in another video. Happy prompting. you END_TEXT_MARKER


### Download Transcription as txt file

In [ ]:
for i, chunk in enumerate(chunks):
  # Se escribe el contenido en un archivo txt
  with open(f'{filename} - transcription({i+1}).txt', 'w') as file:
      file.write(chunk)

In [ ]:
for i, chunk in enumerate(chunks):
  # Se descarga el txt
  files.download(f'{filename} - transcription({i+1}).txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Download Transcription as json file

In [ ]:
import json

# Se guarda el texto completo
with open(f'{filename} - transcription.json', 'w') as file:
    json.dump(text_to_traduce, file, indent=4)

# Se descarga el json
files.download(f'{filename} - transcription.json')